In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
housing_file = "housing.csv"
housing_file_path = ''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename.find(housing_file) != -1:
            housing_file_path = os.path.join(dirname, filename)
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Data preprocessing

### 1.1 Let us load the housing dataset and get some intuition

__*Following implementation is based on the book Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow 3rd Edition by Aurélien Géron*__

In [ ]:
housing = pd.read_csv(housing_file_path)

# print(housing.info())
# print(housing.head())
# print(housing.describe())

### 1.2 Find how many categories of ocean_proximity are there and how many samples in each category

In [ ]:
print(housing["ocean_proximity"].value_counts()) # need to convert this categorical column to numeric equivalent

In [ ]:
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(12, 8))
plt.show()

### 1.3 Let us get test data out of the way before we build any further intuition

#### 1.3.1 Let us process median_income - It should have more impact on the label (median_house_value)

In oder to get test data that properly represents different income categories, instead of random split to get some test data, let us do the following based on histogram of median_income.

median_income can be split into a few ranges and then name each one as one category (like 1, 2, 3...).

We can, infact, make it as a temporary column and use it to select test data from each of the income category, so we have proportional test data from each income category.

In [ ]:
housing['income_category'] = pd.cut(housing['median_income'], bins=[0., 1.5, 3.0, 4.5, 6.0, np.inf], labels=[1, 2, 3, 4, 5])
housing['income_category'].value_counts().sort_index().plot.bar(rot=0, grid=True)
plt.xlabel("Income category")
plt.ylabel("Number of districts")
plt.show()

#### 1.3.2 Use stratified split to get proper distribution of data set into train and test

Manually splitting is one option, gives better understanding of the underlying process...
However, if that understanding is already gained, then we can just use train_test_split with stratify on income_category

In [ ]:
from sklearn.model_selection import train_test_split

strat_train_set, strat_test_set = train_test_split(housing, test_size=0.2, stratify=housing['income_category'], random_state=42)

# let us see how well stratification worked
print(strat_test_set['income_category'].value_counts() / len(strat_test_set))

In [ ]:
# from now on we don't need income_category... let us drop it
for set_ in (strat_train_set, strat_test_set):
    set_.drop('income_category', axis=1, inplace=True)
    
# make strat_train_set as housing (now, housing will only be with train data (without test data, i.e. strat_test_data))
housing = strat_train_set.copy()

#### 1.3.3 Let us get more intuition on the data (without strat_test_data)

Let us plot median_house_value (label to predict), population against longitude and latitude.

Then we will see how median_house_value correlates with the features

In [ ]:
housing.plot(kind='scatter', x="longitude", y='latitude', grid=True,
            s=housing['population'] / 100, label='population',
            c='median_house_value', cmap='jet', colorbar=True,
            legend=True, sharex=False, figsize=(12,8))
plt.show()

From the above plot, median_house_value is high near south west, perhaps, near coastal area

In [ ]:
corr_matrix = housing.corr(numeric_only=True) # only checks for linear correlation...
print(corr_matrix['median_house_value'].sort_values(ascending=False))

From the above correlation matrix value, median_income has the highest correlation (0.687151) with median_house_value

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ['median_house_value', 'median_income', 'total_rooms', 'housing_median_age']
scatter_matrix(housing[attributes], figsize=(12, 8))
plt.show()

In [ ]:
# median_income seems to have interesting correlation with median_house_value
housing.plot(kind="scatter", x='median_income', y='median_house_value', alpha=0.2, grid=True)
plt.show()

#### total rooms, total_bedrooms, population and households are not useful individually by themselves

Let us create a new features based on these existing features

In [ ]:
housing['rooms_per_house'] = housing['total_rooms'] / housing['households']
housing['bedrooms_ratio'] = housing['total_bedrooms'] / housing['total_rooms']
housing['people_per_house'] = housing['population'] / housing['households']

corr_matrix = housing.corr(numeric_only=True) # only checks for linear correlation... sometimes can be misleading
print(corr_matrix['median_house_value'].sort_values(ascending=False))

#### This much idea and general intuition about the data set should be good enough...
Let us start with data processing for ML

#### 1.3.4 Getting ready for ML

Get X (features) and y (label/target)

In [ ]:
housing = strat_train_set.drop('median_house_value', axis=1) # X
housing_labels = strat_train_set['median_house_value'].copy() # y

#### Let us fix total_bedrooms having null values - Enter Imputer!

We can use median value to fill null values

In [ ]:
# # Let us fix total_bedrooms having null values - Enter Imputer

# from sklearn.impute import SimpleImputer # other imputers include KNNImputer, IterativeImputer
# imputer = SimpleImputer(strategy='median') # other strategies are ("mean"), ("most_frequent"), ("constant", fill_value=...)

# housing_num = housing.select_dtypes(include=[np.number]) # select only numeric columns
# imputer.fit(housing_num)
# # print(imputer.statistics_)
# X = imputer.transform(housing_num)
# # print(X)

In [ ]:
# # let us learn about handling categorical columns
# housing_cat = housing[['ocean_proximity']]

# # OrdinalEncoder - ordinal encoding might mislead ML algorithm by paying more attention than needed to the ordinal values
# # OneHotEncoder is a better option (definitely preferred than pandas' get_dummies())
# from sklearn.preprocessing import OneHotEncoder
# cat_encoder = OneHotEncoder()
# housing_cat_1hot = cat_encoder.fit_transform(housing_cat)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# std_scaler = StandardScaler()
# housing_num_std_scaled = std_scaler.fit_transform(housing_num)

##### Feature scaling has more to learn like handling heavy tail then replacing them with its logarithm. Let us do that later...

##### Sometimes even the target values need to be transformed (scaled) like using one's logarithm values
##### then predicted values too would be logarithm values... inverse_tranform() would be useful to determine the actual/intended predicted value

In [ ]:
# from sklearn.compose import TransformedTargetRegressor
# from sklearn.linear_model import LinearRegression

# model = TransformedTargetRegressor(LinearRegression(), transformer=StandardScaler())
# model.fit(housing[['median_income']], housing_labels)
# some_new_data = housing[["median_income"]].iloc[:5]
# predictions = model.predict(some_new_data)
# print(predictions)

##### Let us visualize population

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 3), sharey=True)
housing["population"].hist(ax=axs[0], bins=50)
housing["population"].apply(np.log).hist(ax=axs[1], bins=50)
axs[0].set_xlabel("Population")
axs[1].set_xlabel("Log of population")
axs[0].set_ylabel("Number of districts")

plt.show()

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import rbf_kernel

class ClusterSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, n_clusters=10, gamma=1.0, random_state=None):
        self.n_clusters = n_clusters
        self.gamma = gamma
        self.random_state = random_state

    def fit(self, X, y=None, sample_weight=None):
        self.kmeans_ = KMeans(self.n_clusters, n_init=10,
                              random_state=self.random_state)
        self.kmeans_.fit(X, sample_weight=sample_weight)
        return self  # always return self!

    def transform(self, X):
        return rbf_kernel(X, self.kmeans_.cluster_centers_, gamma=self.gamma)
    
    def get_feature_names_out(self, names=None):
        return [f"Cluster {i} similarity" for i in range(self.n_clusters)]

In [ ]:
# from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector#, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# num_attribs = ["longitude", "latitude", "housing_median_age", "total_rooms",
#                "total_bedrooms", "population", "households", "median_income"]
# cat_attribs = ["ocean_proximity"]

# num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

cat_pipeline = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore"))

# preprocessing = ColumnTransformer([
#     ("num", num_pipeline, num_attribs),
#     ("cat", cat_pipeline, cat_attribs),
# ])

# preprocessing = make_column_transformer(
#                 (num_pipeline, make_column_selector(dtype_include=np.number)),
#                 (cat_pipeline, make_column_selector(dtype_include=object)))

# housing_prepared = preprocessing.fit_transform(housing)

In [ ]:
from sklearn.preprocessing import FunctionTransformer

def column_ratio(X):
    return X[:, [0]] / X[:, [1]]

def ratio_name(function_transformer, feature_names_in):
    return ['ratio']

def ratio_pipeline():
    return make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(column_ratio, feature_names_out=ratio_name),
    StandardScaler())

log_pipeline = make_pipeline(
    SimpleImputer(strategy="median"),
    FunctionTransformer(np.log, feature_names_out="one-to-one"),
    StandardScaler())

cluster_similarity = ClusterSimilarity(n_clusters=10, gamma=1., random_state=42)
default_num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())

from sklearn.compose import ColumnTransformer

preprocessing = ColumnTransformer([
    ("bedrooms", ratio_pipeline(), ["total_bedrooms", "total_rooms"]),
    ("rooms_per_house", ratio_pipeline(), ["total_rooms", "households"]),
    ("people_per_house", ratio_pipeline(), ["population", "households"]),
    ("log", log_pipeline, ["total_bedrooms", "total_rooms", "population", "households", "median_income"]),
    ("geo", cluster_similarity, ["latitude", "longitude"]),
    ("cat", cat_pipeline, make_column_selector(dtype_include=object)),    
], remainder=default_num_pipeline)

In [ ]:
from sklearn.linear_model import LinearRegression

linear_model = make_pipeline(preprocessing, LinearRegression())
linear_model.fit(housing, housing_labels)

linear_housing_predictions = linear_model.predict(housing)

print(linear_housing_predictions[:5].round(-2))
print(housing_labels.iloc[:5].values)

In [ ]:
from sklearn.metrics import mean_squared_error
linear_rmse = mean_squared_error(housing_labels, linear_housing_predictions, squared=False)
print(linear_rmse)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_regressor = make_pipeline(preprocessing, DecisionTreeRegressor(random_state=42))
tree_regressor.fit(housing, housing_labels)

tree_housing_predictions = tree_regressor.predict(housing)

tree_rmse = mean_squared_error(housing_labels, tree_housing_predictions, squared=False)
print(tree_rmse)

In [ ]:
from sklearn.model_selection import cross_val_score

tree_rmses = -1 * cross_val_score(tree_regressor, housing, housing_labels, scoring="neg_root_mean_squared_error", cv=10)
print(pd.Series(tree_rmses).describe())

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_regressor = make_pipeline(preprocessing, RandomForestRegressor(random_state=42))
randomforest_rmses = -1 * cross_val_score(forest_regressor, housing, housing_labels, scoring='neg_root_mean_squared_error', cv=10)
print(pd.Series(tree_rmses).describe())

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

full_pipeline = Pipeline([('preprocessing', preprocessing),
                         ('random_forest', RandomForestRegressor(random_state=42))])

param_grid = [{'preprocessing__geo__n_clusters': [5, 8, 10], 'random_forest__max_features': [4, 6, 8]}, # 3 * 3 = 9
             {'preprocessing__geo__n_clusters': [10, 15], 'random_forest__max_features': [6, 8, 10]}] # 2 * 3 = 6

# (9 + 6) * cv (3) = 45 rounds of training
grid_search = GridSearchCV(full_pipeline, param_grid, cv=3, scoring='neg_root_mean_squared_error')
grid_search.fit(housing, housing_labels)

In [ ]:
print(grid_search.best_params_)
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by='mean_test_score', ascending=False, inplace=True)
print(cv_res.head())

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {'preprocessing__geo__n_clusters': randint(low=3, high=50),
                 'random_forest__max_features': randint(low=2, high=20)}

random_search = RandomizedSearchCV(full_pipeline, param_distributions=param_distribs, n_iter=10, cv=3,
                                   scoring='neg_root_mean_squared_error', random_state=42)

random_search.fit(housing, housing_labels)

# finally (hopefully!)
final_model = random_search.best_estimator_
feature_importances = final_model['random_forest'].feature_importances_
print(feature_importances.round(2))

print(sorted(zip(feature_importances, final_model['preprocessing'].get_feature_names_out()), reverse=True))

In [ ]:
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set['median_house_value'].copy()

final_predictions = final_model.predict(X_test)
final_rmse = mean_squared_error(y_test, final_predictions, squared=False)
print(final_rmse)

In [ ]:
from scipy import stats
confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1, loc=squared_errors.mean(), scale=stats.sem(squared_errors)))

In [ ]:
import joblib
joblib.dump(final_model, 'california_housing_model.pkl')

In [ ]:
# extra code – excluded for conciseness
# from sklearn.cluster import KMeans
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.metrics.pairwise import rbf_kernel

# def column_ratio(X):
#     return X[:, [0]] / X[:, [1]]

# #class ClusterSimilarity(BaseEstimator, TransformerMixin):
# #    [...]

# final_model_reloaded = joblib.load("california_housing_model.pkl")

# new_data = housing.iloc[:5]  # pretend these are new districts
# predictions = final_model_reloaded.predict(new_data)

In [ ]:
# predictions